### **네이버 뉴스 기반 QA(Question-Answering) 챗봇**

In [97]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import bs4 # 파싱용
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [106]:
# 웹 문서 로드 
loader = WebBaseLoader(
    
    web_paths = ('https://n.news.naver.com/mnews/article/029/0002943263',
                 'https://n.news.naver.com/mnews/article/469/0000855476'),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            'div',
            attrs = {'class' : ['media_end_head_title', 'newsct_article _article_body']}
                     
            )
    ),
    header_template={
        "User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)
  
docs = loader.load()

print(docs[0].page_content[:300])


국민연금 모수개혁안 논쟁 가열… 野원외 "성과 인정해야"


여권 이어 민주 인사들도 반박김성주 "'세대간 전쟁' 아니다"박용진 "개혁동력 확장해야"최근 여야 합의로 국회를 통과한 국민연금 모수개혁안을 두고 정치권에서 논쟁이 이어지고 있다. 특히 한동훈 전 국민의힘 대표, 유승민 전 국민의힘 의원 등 여당 '대권 잠룡'들이 적극적으로 비판의 목소리를 내고 있는 가운데 더불어민주당 원외 인사들이 반박에 나서며 논쟁이 불붙는 양상이다.김성주 더불어민주당 전 의원은 24일 자신의 페이스북에 글을 올려 "세대 간 연대로 성립하는 연금제도


In [107]:
# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
split_docs = text_splitter.split_documents(docs)

# Embedding Model
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

# Vector Store
vector_store = Chroma.from_documents(documents=split_docs, embedding=embeddings)

# Retriever 
retriever = vector_store.as_retriever()

In [108]:
# Hub에 있는 Pormpt Load
prompt = hub.pull('teddynote/rag-prompt')
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': 'f42521f58f89165b1784695ac09ff4fb7b750dd7227b0aaf952521e91f629e88'} template="You are an AI assistant specializing in Question-Answering (QA) tasks within a Retrieval-Augmented Generation (RAG) system. \nYour primary mission is to answer questions based on provided context or chat history.\nEnsure your response is concise and directly addresses the question without any additional narration.\n\n###\n\nYour final answer should be written concisely (but include important numerical values, technical terms, jargon, and names), followed by the source of the information.\n\n# Steps\n\n1. Carefully read and understand the context provided.\n2. Identify the key information related to the question within the context.\n3. Formulate a concise answer based on the relevant information.\n4. Ensure your final answer directly addresses the

In [ ]:
# LLM model 
llm = ChatGroq(model_name = 'llama3-70b-8192')


# Chain 생성
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)

user = '''
1. 산불은 몇건이 발생했나요? 
2. 산불로 인한 피해에는 어떤것들이 있나요
3. 어느 지역에 산불이 발생했나요
'''

response = chain.stream(user)

for chunk in response:
    print(chunk, end = '', flush = True)

Here are the answers to the user's questions:

1. 산불은 몇건이 발생했나요?
5건

**Source**
- https://n.news.naver.com/mnews/article/469/0000855476

2. 산불로 인한 피해에는 어떤것들이 있나요
문화유산 5건 (울주 목도 상록수림, 운화리성지, 의성 사촌리 가로숲, 의성 고운사 석조여래좌상, 목조아미타여래좌상)

**Source**
- https://n.news.naver.com/mnews/article/469/0000855476
- https://n.news.naver.com/mnews/article/469/0000855476

3. 어느 지역에 산불이 발생했나요
경북 의성군, 울산 울주군

**Source**
- https://n.news.naver.com/mnews/article/469/0000855476
- https://n.news.naver.com/mnews/article/469/0000855476